分析过程参考：https://zhuanlan.zhihu.com/p/103866380

## 一、导入数据并清洗

In [1]:
import pandas as pd

df = pd.read_csv("./data/UserBehavior_part.csv")
df.head()

,customer_id,product_id,product_category_id,behavior_type,time_stamp
0,726027,1376740,2892802,pv,1512224887
1,489422,3911911,1265358,pv,1511958690
2,649519,1721829,2440115,pv,1512110279
3,556136,4152590,903809,pv,1511747452
4,1009769,3227805,4719814,pv,1511695257


In [2]:
# 查看数据长度
len(df)

1000000

In [3]:
# 查看各列的长度，看是否有缺失
df.count()

customer_id            1000000
product_id             1000000
product_category_id    1000000
behavior_type          1000000
time_stamp             1000000
dtype: int64

In [4]:
# 删除其中的重复行，再次查看长度
df.drop_duplicates(inplace=True)
df.count()

customer_id            1000000
product_id             1000000
product_category_id    1000000
behavior_type          1000000
time_stamp             1000000
dtype: int64

In [5]:
# 对时间戳进行处理，转换为标准的时间格式
df["time_stamp"] = pd.to_datetime(df["time_stamp"], unit="s")

# 对时间与日期进行拆分
df.insert(loc=df.shape[1], column="date", value=df["time_stamp"].dt.date)
df.insert(loc=df.shape[1], column="time", value=df["time_stamp"].dt.time)
df.drop(labels="time_stamp", axis=1, inplace=True)

df.head()

,customer_id,product_id,product_category_id,behavior_type,date,time
0,726027,1376740,2892802,pv,2017-12-02,14:28:07
1,489422,3911911,1265358,pv,2017-11-29,12:31:30
2,649519,1721829,2440115,pv,2017-12-01,06:37:59
3,556136,4152590,903809,pv,2017-11-27,01:50:52
4,1009769,3227805,4719814,pv,2017-11-26,11:20:57


In [6]:
# 由于数据分析需要从时间 17-11-25 至 17-12-03 之间，因此需要剔除不符合这时间跨度的数据
from datetime import datetime

start_date = datetime.strptime("20171125", "%Y%m%d").date()
end_date = datetime.strptime("20171203", "%Y%m%d").date()

df = df[(df["date"]>=start_date) & (df["date"]<=end_date)]

len(df)

987856

## 二、对数据进行统计分析

# 2.1 观察各类行为的数量及比例

In [7]:
# 分析各类用户行为的数量，包括 pv=点击、buy=购买、cart=加车、fav=收藏
behavior_df = df.groupby("behavior_type").count()["customer_id"].to_frame()  # 将 Series 转化为 DataFrame，才能使用下面的 insert 方法
behavior_df.columns = ["count"]
behavior_df.insert(loc=behavior_df.shape[1], column="ratio", value=behavior_df["count"].div(len(df)))

behavior_df

,count,ratio
behavior_type,,
buy,20083,0.020330
cart,54555,0.055226
fav,28200,0.028547
pv,885018,0.895898


结论：从结果可见，将近 9 成的用户仅进行了点击，仅有 2% 的行为为购买。由此可以得出商品的点击转化率较低（点击转化率=购买客户数/点击（访问）客户数），主要原因可能是淘宝的推荐机制不够完善。

# 2.2 观察各类行为对应最多的 5 件商品

In [8]:
# 观察点击量最多的 5 个商品类目
pv_df = df[df["behavior_type"]=="pv"].groupby("product_category_id").count()["customer_id"].to_frame()
pv_df.columns = ["count"]
pv_df.sort_values("count", inplace=True, ascending=False)  # 降序排列

pv_df.head()

,count
product_category_id,
4756105,44240
2355072,31307
4145813,30992
3607361,29253
982926,27725


In [9]:
# 观察购买量最多的 5 个商品类目
buy_df = df[df["behavior_type"]=="buy"].groupby("product_category_id").count()["customer_id"].to_frame()
buy_df.columns = ["count"]
buy_df.sort_values("count", inplace=True, ascending=False)  # 降序排列

buy_df.head()

,count
product_category_id,
1464116,337
2735466,320
4756105,301
2885642,299
4145813,288


In [10]:
# 观察加车量最多的 5 个商品类目
cart_df = df[df["behavior_type"]=="cart"].groupby("product_category_id").count()["customer_id"].to_frame()
cart_df.columns = ["count"]
cart_df.sort_values("count", inplace=True, ascending=False)  # 降序排列

cart_df.head()

,count
product_category_id,
4756105,2209
4145813,1770
982926,1543
4801426,1305
2355072,1144


In [11]:
# 观察收藏量最多的 5 个商品类目
fav_df = df[df["behavior_type"]=="fav"].groupby("product_category_id").count()["customer_id"].to_frame()
fav_df.columns = ["count"]
fav_df.sort_values("count", inplace=True, ascending=False)  # 降序排列

fav_df.head()

,count
product_category_id,
4756105,1384
4145813,1054
982926,887
2355072,882
3607361,737


结论：从上述统计结果来看，点击量最多的 5 类商品中：有 4 类存在于 5 类加车最多的商品类目中；且该 5 类刚好全部都是收藏最多的商品类目；但是其中只有 2 类存在于 5 类最多购买的商品类目中。说明，适当地增加商品的曝光率（点击数），有利于增加用户的购买欲望（收藏、加车），但是不一定能促进最终的购买行为（也可能只是因为点击到购买之间的时间滞后导致的），可能淘宝的推荐并没有符合用户的预期。